will later move this into function script

In [ ]:
import pandas as pd
import re

In [5]:
def clean_column(col: str) -> str:
    """
    Clean a single column name by:
      - Stripping whitespace and replacing any non-standard quotes.
      - Using regex to check for per-process metric patterns (e.g., gpu_energy_process_0).
      - Otherwise, stripping off any messy prefixes using a known list of tokens.
    
    If no known token is found, the original column name is returned.
    """
    # Normalize the column string: remove extra whitespace and fix common issues with quotes.
    col = col.strip()
    col = col.replace("“", "\"").replace("”", "\"")
    
    # First, check if it is a per-process metric column.
    per_process_patterns = [
        r'(cpu_power_process_\d+)',
        r'(gpu_power_process_\d+)',
        r'(ram_power_process_\d+)',
        r'(cpu_energy_process_\d+)',
        r'(gpu_energy_process_\d+)',
        r'(ram_energy_process_\d+)',
        r'(total_energy_kwh_process_\d+)',
        r'(total_energy_joules_process_\d+)'
    ]
    for pattern in per_process_patterns:
        match = re.search(pattern, col)
        if match:
            return match.group(1)
    
    # For non-per-process columns, search for a known token
    tokens = [ 
        "config_name", "experiment_id", "date_time", "model", "is_encoder_decoder",
        "task_type", "available_gpu_count", "gpu_model", "available_cpu_count", "cpu_model",
        "os", "python_version", "country", "region", "fsdp_use_orig_params", "fsdp_cpu_offload",
        "sharding_strategy", "distributed_type", "num_processes", "max_input_tokens", "max_output_tokens",
        "number_input_prompts", "decode_token_to_text", "decoder_temperature", "decoder_top_k", "decoder_top_p",
        "query_rate", "latency_simulate", "latency_delay_min", "latency_delay_max", "latency_simulate_burst",
        "latency_burst_interval", "latency_burst_size", "fp_precision", "quantization", "load_in_8bit",
        "load_in_4bit", "cached_flops_for_quantised_models", "batch_size___fixed_batching", "adaptive_batching",
        "adaptive_max_tokens", "max_batch_size___adaptive_batching", "inference_type", "backend", "total_params",
        "architecture", "total_input_tokens", "total_generated_tokens", "total_inference_time_sec", 
        "average_latency_ms_per_batch", "throughput_queries_per_sec", "throughput_tokens_per_sec", "flops",
        "gpu_current_memory_allocated_bytes", "gpu_max_memory_allocated_bytes", "gpu_current_memory_reserved_bytes",
        "gpu_max_memory_reserved_bytes", "gpu_utilization_percent", "cpu_usage_percent", "cpu_memory_usage_bytes",
        "cpu_power_avg", "gpu_power_avg", "ram_power_avg", "cpu_energy_total", "gpu_energy_total", "ram_energy_total",
        "total_energy_kwh", "total_energy_joules", "tokens_per_joule", "joules_per_token", "flops_per_joule", "joules_per_flop",
        "per-process_emissions"
    ]
    
    for token in tokens:
        # Check if the token exists anywhere in the column name.
        if token in col:
            idx = col.find(token)
            return col[idx:]
    
    # If no known token is found, return the cleaned (normalized) column.
    return col

def clean_and_reorder_columns(df: pd.DataFrame, desired_order: list) -> pd.DataFrame:
    """
    Clean DataFrame columns by:
      1. Renaming each column to remove extraneous prefixes.
      2. Reordering columns into the order specified by 'desired_order'.
         Any columns not explicitly mentioned will be appended at the end.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with messy, flattened column names.
        desired_order (list): List of column names (after cleaning) indicating the preferred ordering.
        
    Returns:
        pd.DataFrame: DataFrame with cleaned and reordered columns.
    """
    # Build mapping from original column names to cleaned names.
    mapping = {}
    for col in df.columns:
        new_name = clean_column(col)
        mapping[col] = new_name

    # Optionally, you might want to print the mapping for debugging:
    # for orig, new in mapping.items():
    #     print(f"Original: '{orig}' -> Cleaned: '{new}'")
    
    # Rename columns in the DataFrame.
    df = df.rename(columns=mapping)
    
    # Reorder columns: first, the ones matching the desired order.
    ordered_cols = [col for col in desired_order if col in df.columns]
    # Then append any remaining columns.
    remaining_cols = [col for col in df.columns if col not in desired_order]
    final_order = ordered_cols + remaining_cols
    
    return df[final_order]

desired_order = [
    "config_name",
    "experiment_id",
    "date_time",
    "model",
    "is_encoder_decoder",
    "task_type",
    "available_gpu_count",
    "gpu_model",
    "available_cpu_count",
    "cpu_model",
    "os",
    "python_version",
    "country",
    "region",
    "fsdp_use_orig_params",
    "fsdp_cpu_offload",
    "sharding_strategy",
    "distributed_type",
    "num_processes",
    "max_input_tokens",
    "max_output_tokens",
    "number_input_prompts",
    "decode_token_to_text",
    "decoder_temperature",
    "decoder_top_k",
    "decoder_top_p",
    "query_rate",
    "latency_simulate",
    "latency_delay_min",
    "latency_delay_max",
    "latency_simulate_burst",
    "latency_burst_interval",
    "latency_burst_size",
    "fp_precision",
    "quantization",
    "load_in_8bit",
    "load_in_4bit",
    "cached_flops_for_quantised_models",
    "batch_size___fixed_batching",
    "adaptive_batching",
    "adaptive_max_tokens",
    "max_batch_size___adaptive_batching",
    "inference_type",
    "backend",
    "total_params",
    "architecture",
    "total_input_tokens",
    "total_generated_tokens",
    "total_inference_time_sec",
    "average_latency_ms_per_batch",
    "throughput_queries_per_sec",
    "throughput_tokens_per_sec",
    "flops",
    "gpu_current_memory_allocated_bytes",
    "gpu_max_memory_allocated_bytes",
    "gpu_current_memory_reserved_bytes",
    "gpu_max_memory_reserved_bytes",
    "gpu_utilization_percent",
    "cpu_usage_percent",
    "cpu_memory_usage_bytes",
    # Per-process metrics:
    "cpu_power_process_0", "cpu_power_process_1", "cpu_power_process_2", "cpu_power_process_3",
    "gpu_power_process_0", "gpu_power_process_1", "gpu_power_process_2", "gpu_power_process_3",
    "ram_power_process_0", "ram_power_process_1", "ram_power_process_2", "ram_power_process_3",
    "cpu_energy_process_0", "cpu_energy_process_1", "cpu_energy_process_2", "cpu_energy_process_3",
    "gpu_energy_process_0", "gpu_energy_process_1", "gpu_energy_process_2", "gpu_energy_process_3",
    "ram_energy_process_0", "ram_energy_process_1", "ram_energy_process_2", "ram_energy_process_3",
    "total_energy_kwh_process_0", "total_energy_kwh_process_1", "total_energy_kwh_process_2", "total_energy_kwh_process_3",
    "total_energy_joules_process_0", "total_energy_joules_process_1", "total_energy_joules_process_2", "total_energy_joules_process_3",
    # Global averages and totals:
    "cpu_power_avg",
    "gpu_power_avg",
    "ram_power_avg",
    "cpu_energy_total",
    "gpu_energy_total",
    "ram_energy_total",
    "total_energy_kwh",
    "total_energy_joules",
    "tokens_per_joule",
    "joules_per_token",
    "flops_per_joule",
    "joules_per_flop",
    "per-process_emissions"
]


In [8]:
def inspect_results(name, desired_order):
    # Build filenames
    input_file = f"results/{name}_results.csv"
    output_file = f"results/cleaned_{name}.csv"
    
    # Load
    df = pd.read_csv(input_file)
    
    # Clean and reorder
    df_cleaned = clean_and_reorder_columns(df, desired_order)
    
    # Save cleaned version
    df_cleaned.to_csv(output_file, index=False)
    
    # Set pandas display options
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    # Display transposed cleaned DataFrame
    display(df_cleaned.T)
    
    # Return in case you want it
    return df_cleaned

df_controlled = inspect_results('controlled', desired_order)

,0,1,2,3,4,5,6,7,8,9,10
config_name,num_processes_1,num_processes_2,num_processes_3,num_processes_4,batching_1,batching_2,batching_4,batching_8,batching_16,batching_32,batching_64
experiment_id,12,13,14,15,16,17,18,19,20,21,22
experiment_id,12,13,14,15,16,17,18,19,20,21,22
date_time,"April 08, 2025 at 03:33:17 PM","April 08, 2025 at 03:33:53 PM","April 08, 2025 at 03:34:31 PM","April 08, 2025 at 03:35:12 PM","April 08, 2025 at 03:36:18 PM","April 08, 2025 at 03:37:11 PM","April 08, 2025 at 03:37:57 PM","April 08, 2025 at 03:38:41 PM","April 08, 2025 at 03:39:20 PM","April 08, 2025 at 03:39:55 PM","April 08, 2025 at 03:40:36 PM"
model,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor
model,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB
model,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0
is_encoder_decoder,False,False,False,False,False,False,False,False,False,False,False
task_type,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation
available_gpu_count,4,4,4,4,4,4,4,4,4,4,4


In [9]:
df_text_generation = inspect_results('text_generation', desired_order)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
config_name,precis_float32_quant_False_quant8_False_quant4_False,precis_float16_quant_False_quant8_False_quant4_False,precis_float16_quant_True_quant8_True_quant4_False,precis_float16_quant_True_quant8_False_quant4_True,decoding_greedy_decoder_temperature_0,decoding_greedy_decoder_temperature_0.7,decoding_greedy_decoder_temperature_1.0,decoding_greedy_decoder_temperature_1.3,decoding_top_k_decoder_top_k_50_decoder_temperature_0,decoding_top_k_decoder_top_k_50_decoder_temperature_0.7,decoding_top_k_decoder_top_k_50_decoder_temperature_1.0,decoding_top_k_decoder_top_k_50_decoder_temperature_1.3,decoding_top_p_decoder_top_p_0.9_decoder_temperature_0,decoding_top_p_decoder_top_p_0.9_decoder_temperature_0.7,decoding_top_p_decoder_top_p_0.9_decoder_temperature_1.0,decoding_top_p_decoder_top_p_0.9_decoder_temperature_1.3,latency_False
experiment_id,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
experiment_id,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
date_time,"April 08, 2025 at 03:41:15 PM","April 08, 2025 at 03:41:53 PM","April 08, 2025 at 03:43:03 PM","April 08, 2025 at 03:43:44 PM","April 08, 2025 at 03:44:24 PM","April 08, 2025 at 03:45:04 PM","April 08, 2025 at 03:45:43 PM","April 08, 2025 at 03:46:23 PM","April 08, 2025 at 03:47:02 PM","April 08, 2025 at 03:47:43 PM","April 08, 2025 at 03:48:22 PM","April 08, 2025 at 03:49:02 PM","April 08, 2025 at 03:49:43 PM","April 08, 2025 at 03:50:25 PM","April 08, 2025 at 03:51:06 PM","April 08, 2025 at 03:51:47 PM","April 08, 2025 at 03:52:27 PM"
model,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor,AMD EPYC 7742 64-Core Processor
model,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB,4 x NVIDIA A100-PCIE-40GB
model,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0,TinyLlama/TinyLlama-1.1B-Chat-v1.0
is_encoder_decoder,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
task_type,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation,text_generation
available_gpu_count,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [ ]:
df_text_generation = inspect_results('text_generation', desired_order)